# MVSep-MDX23 Colab Fork v2.2.2
Colab version of MDX23 algorithm from [MVSEP.COM](https://www.mvsep.com) with some tweaks:
<font size=2>

**v2.2.2**
* D1581 demo model replaced by new InstVocHQ MDXv3 model.

**v2.2.1**
* Added custom weights feature
* Fixed some bugs
* Fixed input: you can use a file or a folder as input now

**v2.2**
* Added MDXv3 compatibility 
* Added MDXv3 demo model D1581 in vocals stem multiband ensemble.
* Added VOC-FT Fullband SRS instead of UVR-MDX-Instr-HQ3.
* Added 2stems feature : output only vocals/instrum (faster processing)
* Added 16bit output format option
* Added "BigShift trick" for MDX models
* Added separated overlap values for MDX, MDXv3 and Demucs
* Fixed volume compensation fine-tuning for MDX-VOC-FT
<br>

**v2.1 (by deton24)**
* Updated with MDX-VOC-FT instead of Kim Vocal 2
<br>

**v2.0**
* Updated with new Kim Vocal 2 & UVR-MDX-Instr-HQ3 models
* Folder batch processing
* Fixed high frequency bleed in vocals
* Fixed volume compensation for MDX models
<br>

Credits:
* [ZFTurbo/MVSep](https://github.com/ZFTurbo/MVSEP-MDX23-music-separation-model)
* Models by [Demucs](https://github.com/facebookresearch/demucs), [Anjok](https://github.com/Anjok07/ultimatevocalremovergui) & [Kimberley Jensen](https://github.com/KimberleyJensen)
* Adaptation & tweaks by [jarredou](https://github.com/jarredou/MVSEP-MDX23-Colab_v2/)
</font>

In [ ]:
#@markdown #Installation
#@markdown *Run this cell to install MVSep-MDX23*
print('Installing... This will take 1 minute...')
%cd /content
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2.git &> /dev/null
%cd /content/MVSEP-MDX23-Colab_v2
!pip install -r requirements.txt &> /dev/null
print('Installation done !')

In [ ]:
#@markdown #Separation
from pathlib import Path
import glob

%cd /content/MVSEP-MDX23-Colab_v2


input = '/content/drive/MyDrive' #@param {type:"string"}
output_folder = '/content/drive/MyDrive/output' #@param {type:"string"}
#@markdown ---
#@markdown <font size=2>*Bigshifts=1 to disable that feature*</font>
BigShifts_MDX = 6 #@param {type:"slider", min:1, max:41, step:1}
overlap_MDX = 0 #@param {type:"slider", min:0, max:0.95, step:0.05}
overlap_MDXv3 = 8 #@param {type:"slider", min:2, max:40, step:2}
#@markdown ---
weight_MDXv3 = 8 #@param {type:"slider", min:0, max:10, step:1}
weight_VOCFT = 3 #@param {type:"slider", min:0, max:10, step:1}
weight_HQ3 = 2 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
overlap_demucs = 0.6 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
output_format = 'PCM_16' #@param ["PCM_16", "FLOAT"]
vocals_instru_only = True #@param {type:"boolean"}
if vocals_instru_only:
    vocals_only = '--vocals_only true'
else:
    vocals_only = ''

#@markdown <font size=2 color=red>Use lower chunk_size value if you have memory errors !</font>
chunk_size = 500000 #@param {type:"slider", min:100000, max:1000000, step:100000}

if Path(input).is_file():
  file_path = input
  filename =  Path(input).stem
  Path(output_folder,filename).mkdir(parents=True, exist_ok=True)
  !python inference.py \
        --large_gpu \
        --weight_MDXv3 {weight_MDXv3} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_HQ3 {weight_HQ3} \
        --chunk_size {chunk_size} \
        --input_audio "{file_path}" \
        --overlap_demucs {overlap_demucs} \
        --overlap_MDX {overlap_MDX} \
        --overlap_MDXv3 {overlap_MDXv3} \
        --output_format {output_format} \
        --bigshifts {BigShifts_MDX} \
        --output_folder "{output_folder}/{filename}" \
        {vocals_only}
else:  
  for file_path in sorted(glob.glob(input+"/*"))[:]:
    filename =  Path(file_path).stem
    Path(output_folder,filename).mkdir(parents=True, exist_ok=True)
    !python inference.py \
        --large_gpu \
        --weight_MDXv3 {weight_MDXv3} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_HQ3 {weight_HQ3} \
        --chunk_size {chunk_size} \
        --input_audio "{file_path}" \
        --overlap_demucs {overlap_demucs} \
        --overlap_MDX {overlap_MDX} \
        --overlap_MDXv3 {overlap_MDXv3} \
        --output_format {output_format} \
        --bigshifts {BigShifts_MDX} \
        --output_folder "{output_folder}/{filename}" \
        {vocals_only}